**EXERCISE 1**

The goal of this exercise is to get into some first contact with Python, Pandas data frames, and the many opportunities how to quickly generate static charts from data frames.

For the first exercise, load the provided CSV file, do some data cleaning, and check if your cleaning operation was successful using visualization. For inspiration, browse through some Kaggle notebooks. The original data was downloaded from Kaggle. 

The main purpose of the notebook is to preprocess the CSV file for further visualization. The following steps have to be performed: 

Read the three CSV using Pandas. See the pandas.read_csv documentation to check how to parse the CSV correctly! Merge the two datasets player_data_per_36_min.csv and players.csv into one table. Add a team_name column to your new data frame based on the team_id column retrieving the name from the teams.csv dataset. You should now have a table that includes player info from players.csv, and players performance statistics from player_data_per_36_min.csv, and the team’s name from teams.csv. (3 points)
Take care of missing values. Sci-kit learn provides different data imputation methods. Remove unusable rows or columns, if necessary.  If the player has no current team, replace the empty value with "Retired".
(Hint: You may need to impute before you completely filter your dataframe. You need to make an educated judgment). (3 points)
Create two new tables: one that groups rows based on player_id, another that groups rows based on team_id. (2 points)
Visualize the data (twice). Every submitted notebook should contain at least two visualizations using at least two different Python visualization libraries. One visualization for each of the new tables (aggregated players or aggregated teams). A list of the most wide-spread Python visualization libraries can be found in this article. You must concisely describe and explain each visualization and your decisions in a Markdown field. You will not receive the points for this task if you did not add a description/explanation. (max. 5 points per visualization)
Save the resulting tables (the cleaned player_data_per_36_min and the two new aggregated tables) as CSV. To be sure that the data is correctly saved, you can load it again. You will have to work with these tables for the second exercise. (2 points)
Possible visualizations include, but are not limited to: 

Scatterplots and scatterplot matrices
Parallel coordinates
Radar charts
Bar charts
Box plots and histograms 
Choropleth maps
...

In [40]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

Read data

In [2]:
players = pd.read_csv("data/players.csv")
teams = pd.read_csv("data/teams.csv")
player_data_per_36_min = pd.read_csv("data/player_data_per_36_min.csv")

Check for duplicates

In [3]:
print(f"Duplikate in Player{len(player_data_per_36_min[player_data_per_36_min.duplicated()==True])}")

Duplikate in Player0


In [4]:
player_data_per_36_min.shape

(26674, 27)

In [5]:
players[players.duplicated()==True]

,id,name,full_name,position,height,weight,birth_date,birth_place,retired,player_url,current_team_id,teams,text


In [6]:
players.shape

(4820, 13)

In [7]:
teams[teams.duplicated()==True]

,team_id,team_name,short_name,years,total_games,total_wins,total_losses,win_loss_percentage,champions,other_names,link,active


In [8]:
teams.shape

(52, 12)

In [9]:
df1 = players.merge(player_data_per_36_min, how='right', left_on='id', right_on='player_id')

In [10]:
df2 = df1.merge(teams[['team_name', 'team_id']], how='left', left_on='current_team_id', right_on='team_id')

In [11]:
df2 = df2.replace({np.nan:None})

In [12]:
df2.loc[df2["retired"],"team_name"] = "Retired"

In [22]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26674 entries, 0 to 26673
Data columns (total 42 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_x             26674 non-null  int64 
 1   name             26674 non-null  object
 2   full_name        26674 non-null  object
 3   position         26674 non-null  object
 4   height           26674 non-null  object
 5   weight           26671 non-null  object
 6   birth_date       26673 non-null  object
 7   birth_place      26278 non-null  object
 8   retired          26674 non-null  bool  
 9   player_url       26674 non-null  object
 10  current_team_id  2830 non-null   object
 11  teams            26674 non-null  object
 12  text             26674 non-null  object
 13  id_y             26674 non-null  int64 
 14  player_id        26674 non-null  int64 
 15  season           26674 non-null  object
 16  total_games      26645 non-null  object
 17  games_started    20208 non-null

In [45]:
df2

,id_x,name,full_name,position,height,weight,birth_date,birth_place,retired,player_url,...,drb,trb,ast,stl,blk,tov,pf,pts,team_name,team_id
0,134,Curly Armstrong,Paul Carlyle Armstrong,G-F,5-11,170.0,November 1 1918,"Fort Wayne, Indiana",True,/players/a/armstcu01.html,...,None,None,None,None,None,None,None,None,Retired,None
1,206,Cliff Barker,Clifford Eugene Barker,G,6-2,185.0,January 15 1921,"Yorktown, Indiana",True,/players/b/barkecl01.html,...,None,None,None,None,None,None,None,None,Retired,None
2,221,Leo Barnhorst,Leo Alphonse Barnhorst,F-G,6-4,190.0,May 11 1924,"Indianapolis, Indiana",True,/players/b/barnhle01.html,...,None,None,None,None,None,None,None,None,Retired,None
3,235,Ed Bartels,Edward John Bartels,F,6-5,195.0,October 8 1925,"New York, New York",True,/players/b/barteed01.html,...,None,None,None,None,None,None,None,None,Retired,None
4,235,Ed Bartels,Edward John Bartels,F,6-5,195.0,October 8 1925,"New York, New York",True,/players/b/barteed01.html,...,None,None,None,None,None,None,None,None,Retired,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26669,4814,Marial Shayok,Marial Shayok,G,6-5,196.0,July 26 1995,"Ottawa, Canada",False,/players/s/shayoma01.html,...,10.8,10.8,3.6,0.0,0.0,3.6,0.0,10.8,Utah Jazz,29.0
26670,4817,Jeremiah Martin,Jeremiah Martin,PG,6-3,184.0,January 19 1996,"Memphis, Tennessee",False,/players/m/martije02.html,...,3.3,3.3,3.3,3.3,0.0,0.0,0.0,6.5,Brooklyn Nets,3.0
26671,4818,William Howard,William Howard,F,6-8,205.0,October 25 1993,"Montbrison, France",False,/players/h/howarwi01.html,...,5.5,5.5,2.8,0.0,0.0,2.8,2.8,0.0,Houston Rockets,11.0
26672,4819,Gabe Vincent,Gabriel Nnamdi Vincent,PG,6-3,200.0,June 14 1996,"Modesto, California",False,/players/v/vincega01.html,...,2.6,2.6,2.6,0.9,0.0,0.0,1.8,10.5,Miami Heat,16.0


In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

In [94]:
birth_places = df2["birth_place"].unique()

In [113]:
birth_states = df2["birth_place"].dropna().apply(lambda x: x.split(','))

In [112]:
birth_states

0          [Fort Wayne,  Indiana]
1            [Yorktown,  Indiana]
2        [Indianapolis,  Indiana]
3           [New York,  New York]
4           [New York,  New York]
                   ...           
26669           [Ottawa,  Canada]
26670       [Memphis,  Tennessee]
26671       [Montbrison,  France]
26672      [Modesto,  California]
26673            [Dallas,  Texas]
Name: birth_place, Length: 26278, dtype: object

In [84]:
all_states = birth_states.unique()

In [79]:
US_states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
    'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
    'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
    'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
    'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
    'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
    'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
    'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
    'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
    'West Virginia', 'Wisconsin', 'Wyoming'
]


In [86]:
all_states[pd.Series(all_states).isin(states)]

array([], dtype=object)

In [50]:
len(birth_places)

1611